# 装pycocotools的几种方式

In [ ]:
# 安装方式1
!pip install -U git+https://gitee.com/fqpang-ncut_fqpang/cocoapi.git   #子目录是PythonAPI
# !pip install git+https://github.com/philferriere/cocoapi.git         #subdirectory=PythonAPI
# 调用方式
import pycocotools

# 安装方式2
!git clone --recurse https://github.com/philferriere/cocoapi.git
!ls cocoapi/PythonAPI
!pip install -e ./cocoapi/PythonAPI/
# 调用方式1 
import cocoapi.PythonAPI.pycocotools
# 调用方式2
import sys
sys.path.append('./cocoapi/PythonAPI/')
import pycocotools

In [ ]:
import sys
sys.version

# 在输出端新建文件夹

In [ ]:
import os
output_dir = '/kaggle/working/data'   # 只有working子目录创建新文件夹可以显示
os.makedirs(output_dir,exist_ok=True)

# 从原图中取15张图

In [ ]:
import numpy as np
from PIL import Image
import math

imgs_test = np.load('../input/cancer-inst-segmentation-and-classification/Images/images.npy') #读入image.npy
# masks_test = np.load('../input/cancer-inst-segmentation-and-classification/Masks/masks.npy') #读入mask.npy
images = Image.fromarray(np.uint8(imgs_test/float(math.pow(2,16)-1)*255))
Image.open(images[0])


# masks = Image.fromarray(masks_test)
# for i in range(0,15):
#     B = images[i, :, :, :]
#     images.save("./data" + str(i) +
#                       "_testResults" + ".png",B)  #保存为png格式，也可将png换位jpg等其他格式

# 打印路径

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))   #打印input文件路径

# mask.npy文件显示

In [ ]:
%matplotlib inline
from skimage.measure import label
import numpy as np
import matplotlib.pyplot as plt

masks = np.load('/kaggle/input/cancer-inst-segmentation-and-classification/Masks/masks.npy',mmap_mode='r')
# mmap_mode='r' ：npy存到disk硬盘

print("type:", type(masks))
print("shape:", masks.shape)                     #显示mask.npy数据类型和维度


for i  in range(0,masks.shape[0]):           #shape有四维（数据量，宽，高，通道数）分别对应0，1，2，3    
    mask = masks[i,:,:,0].astype(np.bool_)   
    plt.imshow(mask)                         #布尔打印二值图
    plt.show()

In [ ]:
 for i in range(0,masks.shape[0]):
 #  mask = masks[0,:,:,ch].astype(np.bool_)
    mask = masks[i,:,:,0]
    print('mask cancer', i, 'labels:', np.unique(mask))   #打印标签
    plt.imshow(mask)                                      #打印用不同颜色标记后的连通域（未分离）
    plt.show()

In [ ]:
from skimage.measure import label
import numpy as np
import matplotlib.pyplot as plt

masks = np.load('/kaggle/input/cancer-inst-segmentation-and-classification/Masks/masks.npy',mmap_mode='r')

# 连通域标记

In [ ]:
for i in range(0,masks.shape[0]):       
#     mask = masks[0,:,:,ch].astype(np.bool_)
    mask = masks[i,:,:,0]                                    #进行二值化
    print('mask:', i) 
    print('labels:', np.unique(mask))                        #确认可以二值化，np.unique找连通域
    labeled_img, num_lb = label(mask, background=0, return_num=True, connectivity=1)  #连通域标记
    print('mask:', i)
    print('labels after connection:', np.unique(labeled_img))#确认可以连通域标记
    print(num_lb)                                            #num_1b是连通域个数
    
    colors = plt.cm.tab20b(np.linspace(0,1,num_lb+1))       #将0，1，2，3归一化到[0,1]标记四个通道颜色(RGB,透明度)
    print(colors)
    lb_masks = np.zeros((mask.shape[0],mask.shape[1],4))    #声明新的图片
    for n in range(num_lb+1):                               #找出n标签的位置
#         print(colors[n,:])
        lb_masks[labeled_img==n] = colors[n,:]              #在n标签对应位置上色     
    plt.imshow(lb_masks)
    plt.show()

# 连通域分离

In [ ]:
for i in range(0,masks.shape[0]):
    mask = masks[i,:,:,0]
#     mask = masks[0,:,:,ch].astype(np.bool_)
    print('mask:', i)
    print('labels:', np.unique(mask))
    labeled_img, num_lb = label(mask, background=0, return_num=True, connectivity=1)
    print('mask:', i)
    print('labels after connection:', np.unique(labeled_img))
    print('num_lb',num_lb)
    

    lb_masks = np.zeros((mask.shape[0],mask.shape[1],num_lb)) #lb_masks已经变为三维，因为batch维已去除
#     for n in range(1,num_lb+1):
    for n in range(1,num_lb+1):    
        print(n)
#         print(colors[n,:])
        lb_masks[labeled_img==n,n-1] = 1       #label是从1开始的，通道数是从0开始的
        plt.imshow(lb_masks[:,:,n-1])         
        plt.show()   

# 显示image.npy

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

images = np.load('/kaggle/input/cancer-inst-segmentation-and-classification/Images/images.npy')#读取image.npy
print("shape:", images.shape)
for i in range(0,images.shape[0]):
    image = images[i,:,:,:].astype(np.uint8)
    plt.imshow(image)                           #显示image彩图
    plt.show()

# =  ==    [:,None,None]

In [ ]:
import numpy as np
x = np.ones((4,4))
# x2 = x[:,:,None]
# print(x)
# print(x2)
label = np.array([1,2])
mask = x == label[:,None,None]
mask

# 建数据集

In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
from skimage.measure import label
 
 
class CancerDataset(torch.utils.data.Dataset):
        
    def __init__(self, images, masks, transforms=None): #定义的全局变量必须加self.
        self.images, self.masks = images, masks
        self.transforms = transforms                               #transform用于数据增强
 
    def __getitem__(self, idx):
        img = self.images[idx,:,:,:]
#         mask = self.masks[idx,:,:,0]
        mask = self.masks[idx,:,:,0].astype(np.bool_)
        print(np.unique(mask))
        mask_ = label(mask, background=0, return_num=True, connectivity=1)  #连通域标记
        print(mask_)
        obj_ids = np.unique(mask_)
        # 去除切片维度，也就是[0]
        obj_ids = obj_ids[1:]
        masks = mask_ == obj_ids[:, None, None]              #把每张图都拿出来放到masks里
 
        #给每个mask加框
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])                           #列是1，行是0
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])          #append：往列表里加元素
 
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
 
 
        image_id = torch.tensor([idx])
      
    
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        
 
 
        if self.transforms is not None:
            img, target = self.transforms(img, target)
 
 
        return img, target
 
 
    def __len__(self):
        return self.images.shape[0]                       #返回值不能是numpy数组，为切片数

# dataloader

In [ ]:
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)

#检验 dataloader
for img,target in data_loader:
    print(img.shape)
    print(target.shape)

## 数据集分割，清空堆栈

In [ ]:
images_split = np.split(images,[1000, 1100, 1200])
print(len(images_split))
images_split.pop()
print(len(images_split))

## 冻结BN层

In [ ]:
#锁BN层
count = 0
for m in model.named_parameters():
    print(m[0])
        m.requires_grad = False
        count +=1
print(count)
optimizer = torch.optim.SGD(filter(lambda p:p.requires_grad , model.parameters()), lr=0.005,
                            momentum=0.9, weight_decay=0.0005)